In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
import os
from tTEM_toolbox import  process_well as pw
link = Path(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\2022\2022SS\ArcGIS\ttem_specs\shapefile\Usgs water well\MartsonTable6.xlsx')
import os
import numpy as np
import tTEM_toolbox as tt
import pandas as pd
import plotly.offline as po
import plotly.graph_objects as go
import plotly.express as px
from pyproj import Transformer
import pykrige as pk
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
#import importlib
#importlib.reload(tt.bootstrap)

In [2]:
os.chdir(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\well_Utah')
well_info = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\well_Utah\USGSdownload\NWISMapperExport.xlsx'
location = r"C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\Gamma\location.csv"
welllog = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\Plot_with_well_log\Well_log.xlsx'
elevation = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\well_Utah\usgs_water_elevation.csv'
ttemname = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\Plot_with_well_log\PD1_I01_MOD.xyz'
ttemname2 = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\Plot_with_well_log\PD22_I03_MOD.xyz'
DOI = r'C:\Users\jldz9\OneDrive - University of Missouri\MST\Code\Python\tTEM_test\Plot_with_well_log\DOID1_DOIStaE.xyz'
time = '2022-3'
def splt_ttem(ttem_df, gwsurface_result):
    def get_distance(group1, group2):
        dis = np.sqrt((group1[0] - group2[0]) ** 2 + (group1[1] - group2[1]) ** 2)
        return dis
    abv_water_table = []
    blw_water_table = []
    ttem_groups = ttem_df.groupby(['UTMX', 'UTMY'])
    well_location = gwsurface_result[['UTMX', 'UTMY']].values
    for name, group in ttem_groups:
        ttem_xy = list(group[['UTMX', 'UTMY']].iloc[0])
        ttem_well_distance = list(map(lambda x: get_distance(ttem_xy, x), well_location))
        match = gwsurface_result.iloc[ttem_well_distance.index(min(ttem_well_distance))]
        elevation = match['water_elevation']
        ttem_abv = group[group['Elevation_End'] >= elevation]
        abv_water_table.append(ttem_abv)
        ttem_blw = group[group['Elevation_End'] < elevation]
        blw_water_table.append(ttem_blw)

    ttem_above = pd.concat(abv_water_table)
    ttem_below = pd.concat(blw_water_table)
    return ttem_above, ttem_below
def plot_bst(dataframe):
    """
    plot bootstrap result

    :param dataframe:
    :return: plotly fig
    """
    fig_hist = go.Figure()
    fig_hist.data = []
    fig_hist.add_trace(go.Histogram(x=dataframe.fine, name='Fine', marker_color='Blue', opacity=0.75))
    fig_hist.add_trace(go.Histogram(x=dataframe.coarse, name='Coarse', marker_color='Red', opacity=0.75))
    if dataframe.mix.sum() == 0:
        print("skip plot mix because there is no data")
    else:
        fig_hist.add_trace(go.Histogram(x=dataframe.mix, name='Mix', marker_color='Green', opacity=0.75))
    fig_hist.update_layout(paper_bgcolor='white',plot_bgcolor='white',font_color='black')

    return fig_hist


In [3]:
usgs_no = pd.read_excel(link,header=None)
df = pd.DataFrame()
for i in range(15):
    number = np.arange(i, usgs_no.shape[0],15)
    tmp = usgs_no[usgs_no.index.isin(number)]
    tmp.reset_index(inplace=True,drop=True)
    df = pd.concat([df,tmp],axis=1,ignore_index=True)
df.columns = ['USGSID','LocalID','Chloride mg/L','Fluoride mg/L','Iron ug/L','Magnesium mg/L','Manganese ug/L','Potassium mg/L','Silica mg/L', 'Sodium mg/L', 'Sulfate mg/L', 'Residue mg/L','Residue 1974 mg/L', 'Nitrate plus nitrite mg/L','Charge balance']
lat = []
long= []
for i in df['USGSID']:
    print(i)
    _, metadata = pw.dl_usgs_water(str(i))
    lat.append(metadata.lat)
    long.append(metadata.long)
df['lat'] = lat
df['long'] = long

380248112413401
380218112422001
380218112421001
380224112424901
380117112422301
380130112430501
380131112431901
380033112423101
375949112431901
375900112435900
375722112462101
375708112461101
375628112465301
375523112451902
375402112455401
375257112483501
375628112485601
375604112495901
375548112500401
375344112521601
375303112512102
375233112475701
375112112551202
375112112542701
375204112530802
375147112530001
375144112512801
375119112494701
375024112542601
374955112484901
374753112485301
375006112554801
374846112560801
374718112572902


In [4]:
df.to_csv(link.parent.joinpath('ParowanTable6.csv'), index=False)

In [5]:
water = tt.main.GWSurface(waterwell=well_info)

C:\Users\jldz9\anaconda3\envs\Research\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning:

Cannot parse header or footer so it will be ignored



375033112561101 Done!
375006112554801 Done!
375112112551202 Done!
375024112542601 Done!
375152112541001 Done!
375152112541001 not able to merge, try to solve the problem by drop duplicates.
375127112530602 Done!
375303112512102 Done!
375338112502801 Done!
375548112500401 Done!
375153112500201 Done!
375153112500201 not able to merge, try to solve the problem by drop duplicates.
375443112495501 Done!
375303112495102 Done!
375233112475701 Done!
375440112500001 Done!
375402112455401 Done!
375523112451902 Done!
375915112442901 Done!
380130112425201 Done!
380033112423101 Done!
All Wells Done!


In [6]:
water_format = water.format(elevation=None, time=time)
water_format13_3 =  water.format(elevation=None, time='2013-3')
water_format12_11 = water.format(elevation=None, time='2012-11')
elevation_diff12_13 = water_format12_11['sl_lev_va2012-11'].subtract(water_format13_3['sl_lev_va2013-3'])
#water_format.dropna(inplace=True)
water_format['water_elevation'] = water_format['sl_lev_va2022-3'].add(elevation_diff12_13)

### Rock trans for centeral Parowan

In [8]:
ttem = tt.main.ProcessTTEM(ttem_path=[ttemname2],
                                   welllog=welllog,
                                   DOI_path=DOI,
                                   layer_exclude=[],
                                    line_exclude=[])
data = ttem.data()
ttem_above_wt, ttem_below_wt = splt_ttem(data, water_format)
bootstrap_above = tt.main.ProcessTTEM(ttem_path=ttem_above_wt, welllog=welllog)
bootstrap_above_result = bootstrap_above.ttem_well_connect(distance=1000)
bootstrap_below = tt.main.ProcessTTEM(ttem_path=ttem_below_wt, welllog=welllog)
bootstrap_below_result = bootstrap_below.ttem_well_connect(distance=1000)
fig_hist_ab = plot_bst(bootstrap_above_result[1])

fig_hist_bl = plot_bst(bootstrap_below_result[1])



In [ ]:

fig_hist_ab.update_xaxes(title=dict(text='Resistivity ohm'))
fig_hist_ab.update_yaxes(title=dict(text='Count'))
fig_hist_bl.update_xaxes(title=dict(text='Resistivity ohm'))
fig_hist_bl.update_yaxes(title=dict(text='Count'))
fig_hist_ab.update_layout(title='Above Water Table')
fig_hist_bl.update_layout(title='Below Water Table')
fig_hist_ab.show(renderer='browser')
fig_hist_bl.show(renderer='browser')

In [ ]:
res_conf_df = tt.bootstrap.packup(bootstrap_above_result[1]['fine'],bootstrap_above_result[1]['mix'], bootstrap_above_result[1]['coarse'] )

res_conf_df_bl = tt.bootstrap.packup(bootstrap_below_result[1]['fine'],bootstrap_below_result[1]['mix'], bootstrap_below_result[1]['coarse'] )
rk_trans_above = tt.Rock_trans.rock_transform(ttem_above_wt,res_conf_df)
rk_trans_below = tt.Rock_trans.rock_transform(ttem_below_wt,res_conf_df_bl)
rk_trans=pd.concat([rk_trans_above,rk_trans_below])
fig_rk_trans = go.Figure(tt.plot.generate_trace(rk_trans,'rock'))
fig_rk_trans.show(renderer='browser')


In [9]:
ttem_above_match, well_match = tt.bootstrap.select_closest(ttem_above_wt,tt.process_well.format_well(welllog, upscale=100),distance=1000,showskip=False)
ttem_below_match, well_match = tt.bootstrap.select_closest(ttem_below_wt,tt.process_well.format_well(welllog, upscale=100),distance=1000,showskip=False)

In [ ]:
ratio_centeral = tt.Rock_trans.pct_count(rk_trans,grain='Fine_grain')
ratio_centeral.to_csv(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\2022\Paper\meeting113022\ratio_centeral.csv')

In [ ]:
fig_rk_trans.write_html(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\2022\2022FS\AGU presentation\centralrk.html')

In [ ]:
ttem_above_match.shape

In [ ]:
ttem_below_match.shape

In [ ]:
res_conf_df

In [ ]:
res_conf_df_bl

### Rocktrans for northern Parowan

In [ ]:
ttem = tt.main.ProcessTTEM(ttem_path=[ttemname],
                                   welllog=welllog,
                                   DOI_path=DOI,
                                   layer_exclude=[],
                                    line_exclude=[0,100,110,120,130,140,150])
data = ttem.data()
ttem_above_wt, ttem_below_wt = splt_ttem(data, water_format)
bootstrap_above = tt.main.ProcessTTEM(ttem_path=ttem_above_wt, welllog=welllog)
bootstrap_above_result = bootstrap_above.ttem_well_connect(distance=1000)
bootstrap_below = tt.main.ProcessTTEM(ttem_path=ttem_below_wt, welllog=welllog)
bootstrap_below_result = bootstrap_below.ttem_well_connect(distance=1000)
fig_hist_ab = plot_bst(bootstrap_above_result[1])
fig_hist_ab.show()
fig_hist_bl = plot_bst(bootstrap_below_result[1])
fig_hist_bl.show()

In [ ]:
ttem = tt.main.ProcessTTEM(ttem_path=[ttemname],
                                   welllog=welllog,
                                   DOI_path=DOI,
                                   layer_exclude=[],
                                    line_exclude=[])
data = ttem.data()
ttem_above_wt, ttem_below_wt = splt_ttem(data, water_format)

In [ ]:
rk_trans_above = tt.Rock_trans.rock_transform(ttem_above_wt,res_conf_df)
rk_trans_below = tt.Rock_trans.rock_transform(ttem_below_wt,res_conf_df_bl)
rk_trans=pd.concat([rk_trans_above,rk_trans_below])
fig_rk_trans = go.Figure(tt.plot.generate_trace(rk_trans,'rock'))
fig_rk_trans.show(renderer='browser')

In [ ]:
fig_rk_trans.write_html(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\2022\2022FS\AGU presentation\northernrk.html')

In [ ]:
ratio_north = tt.Rock_trans.pct_count(rk_trans,grain='Fine_grain')
ratio_north.to_csv(r'C:\Users\jldz9\OneDrive - University of Missouri\MST\2022\Paper\meeting113022\rationorth.csv')